# User-based recommendation on user clusters (based on likes)

## Import

In [130]:
import pandas as pd
import re
from transformers import pipeline

In [131]:
data_reviews = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/reviews_clean.csv")

with pd.option_context('display.max_colwidth', None):
    display(data_reviews.head(1))

uid         profile  anime_uid  \
0  255938  DesolatePsyche      34096   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Preprocessing

In [ ]:
def clean_text(text):
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    text = re.sub("[^A-Za-z]+", " ", text)
    text = text[65:]

    return text

In [150]:
def create_chunks_from_text(text):

    text_splited = re.findall('.{512}', text)
    text_splited.append(text[512 * len(text_splited):])

    return text_splited

In [173]:
df_als_reviews_score = data_reviews[["profile", "anime_uid", "text"]].copy()

In [ ]:
df_als_reviews_score["text"] = df_als_reviews_score["text"].transform(clean_text)
df_als_reviews_score["text"] = df_als_reviews_score["text"].transform(create_chunks_from_text)


In [177]:
df_als_chuncked = df_als_reviews_score[["profile", "anime_uid", "text"]].copy().explode("text")

In [178]:
df_als_chuncked.head(10)

,profile,anime_uid,text
0,DesolatePsyche,34096,t things first My reviews system is explained ...
0,DesolatePsyche,34096,tuff real world anime production and so on Any...
0,DesolatePsyche,34096,nt Of course it has its slice of life side and...
0,DesolatePsyche,34096,I love them Art is cool sometimes artistic som...
0,DesolatePsyche,34096,s of off stereotypes is very large As well cha...
0,DesolatePsyche,34096,tama we have overly serious arcs or fully come...
0,DesolatePsyche,34096,ries maybe because it focused too much moving ...
1,baekbeans,34599,me start off by saying that Made in Abyss is a...
1,baekbeans,34599,e by the mystical vibes exuded by the astoundi...
1,baekbeans,34599,of my seat yet the premise is simple and dige...


In [169]:
with pd.option_context('display.max_colwidth', None):
  display(df_als_reviews_score.head(1))

,profile,anime_uid,text,text_chunked
0,DesolatePsyche,34096,t things first My reviews system is explained on a blog entry Which can be found through my profile I m going to keep this review more of a opinion of Gintama s overall and then this season specific Anyhow What I have always loved regarding Gintama is its content of everything I love the comedy its absurd random can be vile dirty sweet anyhow everything Have laughed countless times in this franchise Also the humor they have also is heavily reference based a k a parodies of different anime shows manga live stuff real world anime production and so on Anyhow comedy parody side of this franchise i absolutely love Now nd side of this show is the serious dramas epic battle shounens and so on There are arcs that are fully comedy arcs that are fully serious and mixtures of both Serious side is usually quite dramatic and managed to somewhat tear me up now and then Whilst the action sequences are absolute bliss as well They are just presented in a really cool manner And extra points when sometime the add artistic element Of course it has its slice of life side and pseudo romance Those are weaker side but ain t the focus But when sometimes those are the focus then they are done in enjoyable method To sum up overall gintama s I love the serious side and the silly side Art and sound voice acting ost op ed is something I loved in all gintamas across the board They just click with me bliss to look at bliss to hear Music op ed specifically are hyped up and upbeat which generally I don t like but gintama is the exception where I love them Art is cool sometimes artistic sometimes simple It has a bit of everything that makes it enjoyable And gore is absolutely satisfying as well well the blood to be specific Well the serious side of stories in this anime are superb in my opinion or the overall idea of story that moves now and then But wouldn t say comedic side of series story is any worse When at times comedy side has story It s usually quite enjoyable Regarding characters I don t have much to say other than that variety stereotypes of off stereotypes is very large As well characters with depth but as well with oddity is as well in great selection Main team Gintoki the lazy dirty slob with superb second side of seriousness and caring about friends allies Shinpachi the poor straight man act sidekick who adds great value to the team Kagura the battle race girl that doesn t act like girl almost at all or in another words a dirty slob girl in a sense Now to this season specific I found it a bit disappointing Mainly because usually in gintama we have overly serious arcs or fully comedic whilst this one had a bit of mixture of both which somewhat ruined the experience Plus the artistic presentation felt this time around a bit lacking Maybe because of another studio who knows But anyhow it didn t feel as superb Rather than gintama it felt as watching just another good battle shounen anime in modern day with simplified animation I mean it definitely was not any bad just didn t have the punch Gintama usually has Might be because it was short series maybe because it focused too much moving on with story It ll be shame to see it finalized in this manner Rest I pretty much loved as usual in Gintama franchise Helpful,"[t things first My reviews system is explained on a blog entry Which can be found through my profile I m going to keep this review more of a opinion of Gintama s overall and then this season specific Anyhow What I have always loved regarding Gintama is its content of everything I love the comedy its absurd random can be vile dirty sweet anyhow everything Have laughed countless times in this franchise Also the humor they have also is heavily reference based a k a parodies of different anime shows manga live s, tuff real world anime production and so on Anyhow comedy parody side of this franchise i absolutely love Now nd side of this show is the serious dramas epic battle shounens and so

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
df_als_reviews_score.to_csv("../data/als_reviews_score.csv", index=False)

## Spark

For now, this part have to be executed in a ad-hoc Jupyter environment with PySpark, following those steps (disclaimer : you need to install and configure PySpark first) :


```shell
pyspark --name anime-recommendation-engine --driver-java-options -Djava.security.manager=allow
```

```python
sc = SparkSession.builder.getOrCreate()
```

[Medium article simple ALS](https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3)

[Medium article advanced ALS](https://medium.com/@brunoborges_38708/recommender-system-using-als-in-pyspark-10329e1d1ee1)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
sc = SparkSession.builder.getOrCreate()

In [ ]:
df_als_is_favorite = pd.read_csv("../data/als_is_favorite.csv")
df_als_reviews_score = pd.read_csv("../data/als_reviews_score.csv")
df_als_favorite_score = pd.read_csv("../data/als_favorite_score.csv")

display(df_als_is_favorite.head(2)) # (216695, 3)
display(df_als_reviews_score.head(2)) # (130519, 3)
display(df_als_favorite_score.head(2)) # (216188, 3)

In [ ]:
def pyspark_df_with_train_test_split(df, test_size, user_col, item_col, rating_col) :

    df_items_counted = df.groupby(user_col)[rating_col].count().reset_index(name="num_items")
    df_merged = df.merge(df_items_counted, on=user_col)

    user_window = Window.partitionBy(user_col).orderBy(col(item_col).desc())

    df_spark = sc.createDataFrame(df_merged)
    df_spark = df_spark.withColumn("num_items_to_mask", (col("num_items") * test_size).cast("int"))
    df_spark = df_spark.withColumn("item_rank", rank().over(user_window))

    indexer = StringIndexer(inputCol=user_col, outputCol=user_col+"_index")

    df_spark = indexer.fit(df_spark).transform(df_spark)

    training = df_spark.filter(col("item_rank") > col("num_items_to_mask"))
    test = df_spark.filter(col("item_rank") <= col("num_items_to_mask"))

    return df_spark, training, test 

In [ ]:
def als_tuning_and_predict(train, test, user_col, item_col, rating_col) :

    als = ALS(userCol=user_col+"_index", itemCol=item_col, ratingCol=rating_col,
            coldStartStrategy="drop", nonnegative=True)

    param_grid = ParamGridBuilder()\
                .addGrid(als.rank, [1, 3, 5])\
                .addGrid(als.maxIter, [5, 10, 15])\
                .addGrid(als.regParam, [.05, .1, .15])\
                .build()
    
    evaluator = RegressionEvaluator(metricName="rmse", labelCol=rating_col, predictionCol="prediction")

    cv = CrossValidator(
            estimator=als,
            estimatorParamMaps=param_grid,
            evaluator=evaluator,
            numFolds=3)
    
    model = cv.fit(train)

    print("Rank: ", model.bestModel.rank)
    print("MaxIter: ", model.bestModel._java_obj.parent().getMaxIter())
    print("RegParam: ", model.bestModel._java_obj.parent().getRegParam())

    predictions = model.bestModel.transform(test)
    rmse = evaluator.evaluate(predictions)
    
    print("RMSE="+str(rmse))

    return model.bestModel

In [ ]:
def create_predictions_pandas_dataframe(model, df_spark, user_col, item_col, rating_col) :
    
    recs = model.recommendForAllUsers(5).toPandas()

    nrecs = recs.recommendations.apply(pd.Series)\
                .merge(recs, right_index = True, left_index = True)\
                .drop(["recommendations"], axis = 1)\
                .melt(id_vars = [user_col+"_index"], value_name = "recommendation")\
                .drop("variable", axis = 1)\
                .dropna()
    nrecs = nrecs.sort_values(user_col+"_index")
    nrecs = pd.concat([nrecs["recommendation"].apply(pd.Series), nrecs[user_col+"_index"]], axis = 1)
    nrecs.columns = [item_col, rating_col, user_col]

    md = df_spark.select(df_spark[user_col],df_spark[user_col+"_index"],df_spark[item_col])
    md = md.toPandas()

    dict1 = dict(zip(md[user_col+"_index"],md[user_col]))

    nrecs[user_col] = nrecs[user_col].map(dict1)
    nrecs = nrecs.sort_values(user_col)
    nrecs.reset_index(drop=True, inplace=True)

    # new = nrecs[[user_col,item_col,rating_col]]
    # new["recommendations"] = list(zip(new[item_col], new[rating_col]))

    # df_recommendations = new[[user_col,"recommendations"]]
    df_recommendations = nrecs[[user_col,item_col]]
    df_recommendations["item_col"] = df_recommendations["item_col"].astype("int64")

    #df_recommendations = df_recommendations["recommendations"].groupby([df_recommendations[user_col]]).apply(list).reset_index()
    df_recommendations = df_recommendations[item_col].groupby([df_recommendations[user_col]]).apply(list).reset_index()

    return df_recommendations

### Is favorite

In [ ]:
df_sp_is_favorite, train, test = pyspark_df_with_train_test_split(df_als_is_favorite, 0.2, "profile", "favorites_anime", "is_favorite")

In [ ]:
model = als_tuning_and_predict(train, test, "profile", "favorites_anime", "is_favorite")

In [ ]:
df_is_favorite_based_reco = create_predictions_pandas_dataframe(model, df_sp_is_favorite, "profile", "favorites_anime", "is_favorite")

display(df_is_favorite_based_reco.head())

In [ ]:
df_is_favorite_based_reco.to_csv("../data/als_is_favorite_based_reco.csv", index=False)

### Reviews score

In [ ]:
df_sp_reviews_score, train, test = pyspark_df_with_train_test_split(df_als_reviews_score, 0.2, "profile", "anime_uid", "score")

In [ ]:
model = als_tuning_and_predict(train, test, "profile", "anime_uid", "score")

In [ ]:
df_score_reviews_based_reco = create_predictions_pandas_dataframe(model, df_sp_reviews_score, "profile", "anime_uid", "score")

display(df_score_reviews_based_reco.head())

In [ ]:
df_score_reviews_based_reco.to_csv("../data/als_reviews_score_based_reco.csv", index=False)

### Is favorite score

In [ ]:
df_sp_favorite_score, train, test = pyspark_df_with_train_test_split(df_als_favorite_score, 0.2, "profile", "favorites_anime", "score")

In [ ]:
model = als_tuning_and_predict(train, test, "profile", "favorites_anime", "score")

In [ ]:
df_is_favorite_score_based_reco = create_predictions_pandas_dataframe(model, df_sp_favorite_score, "profile", "favorites_anime", "score")

display(df_is_favorite_score_based_reco.head())

In [ ]:
df_is_favorite_score_based_reco.to_csv("../data/als_favorite_score_based_reco.csv", index=False)